In [1]:
blobs = open("/kaggle/input/aoc-19-1-fh/test_19.txt", "r").read()[:-1].split('\n\n')
lines = blobs[1].split('\n')

In [2]:
import re
def extract_numbers(text):
    numbers = re.findall(r'\d+\.*\d*', text)
    return numbers

In [3]:
parts = []
for s in lines:
    numbers = extract_numbers(s)
    part = {'x': numbers[0], 'm': numbers[1], 'a': numbers[2], 's': numbers[3]}
    parts.append(part)
lines = blobs[0].split('\n')

In [4]:
def parse_data(input_str):
    pattern = r'([a-zA-Z]+){([^{}]+)}'
    matches = re.findall(pattern, input_str)
    
    result = {}
    for match in matches:
        key = match[0]
        conditions_actions = match[1].split(',')
        conditions = []
        actions = []
        alt = None
        
        for item in conditions_actions:
            if '<' in item or '>' in item:
                condition, action = item.split(':')
                conditions.append(condition)
                actions.append(action)
            else:
                alt = item
        
        result['key'] = key
        for i in range(len(conditions)):
            result[f'condition{i+1}'] = conditions[i]
            result[f'action{i+1}'] = actions[i]
        if alt:
            result['alt'] = alt
    
    return result

In [5]:
workflows = []
for l in lines:
    workflows.append(parse_data(l))

In [6]:
def find_object_by_key(data_list, key):
    for obj in data_list:
        if obj.get('key') == key:
            return obj
    return None

In [7]:
accepted = []
def check_parts(in_string, part):
    x, m, a, s = int(part['x']), int(part['m']), int(part['a']), int(part['s'])
    alt = True
    done = False
    for condition in range((len(in_string)-1)//2):
        new_workflow = ''
        #print(in_string['key'])
        #print(in_string['condition' + str(condition+1)], s)
        if eval(in_string["condition" + str(condition+1)]):
            if in_string["action" + str(condition+1)] == 'A':
                #print('Accepted:', part)
                accepted.append(part)
                done = True
            elif in_string["action" + str(condition+1)] == 'R':
                #print('Rejected:', part)
                done = True
            else:
                #print("Action: ",find_object_by_key(workflows, in_string["action" + str(condition+1)]))
                new_workflow = find_object_by_key(workflows, in_string["action" + str(condition+1)])
            alt = False
            break
    if(alt):
        if in_string["alt"] == 'A':
            #print('Accepted:', part)
            accepted.append(part)
            done = True
        elif in_string["alt"] == 'R':
            #print('Rejected Alt:', part)
            done = True
        else:
            #print("Alt: ",find_object_by_key(workflows, in_string["alt"]))
            new_workflow = find_object_by_key(workflows, in_string["alt"])
    if(done == False):
        check_parts(new_workflow, part)

In [8]:
in_string = find_object_by_key(workflows, 'in')
for part in parts:
    check_parts(in_string, part)

In [9]:
res = 0
for part in accepted:
    res += int(part['x']) + int(part['m']) + int(part['a']) + int(part['s'])

In [10]:
res

19114

# **PART 2**

In [11]:
import re
ll = [x for x in open('/kaggle/input/aoc-19-1-fh/input_19.txt').read().strip().split('\n\n')]
workflow, parts = ll

def ints(s):
	return list(map(int, re.findall(r'\d+', s)))

parts = [ints(l) for l in parts.split("\n")]
workflow = {l.split("{")[0]: l.split("{")[1][:-1] for l in workflow.split("\n")}

def both(ch, gt, val, ranges):
	ch = 'xmas'.index(ch)
	ranges2 = []
	for rng in ranges:
		rng = list(rng)
		lo, hi = rng[ch]
		if gt:
			lo = max(lo, val + 1)
		else:
			hi = min(hi, val - 1)
		if lo > hi:
			continue
		rng[ch] = (lo, hi)
		ranges2.append(tuple(rng))
	return ranges2


def acceptance_ranges_outer(work):
	return acceptance_ranges_inner(workflow[work].split(","))

def acceptance_ranges_inner(w):
	it = w[0]
	if it == "R":
		return []
	if it == "A":
		return [((1, 4000), (1, 4000), (1, 4000), (1, 4000))]
	if ":" not in it:
		return acceptance_ranges_outer(it)
	cond = it.split(":")[0]
	gt = ">" in cond
	ch = cond[0]
	val = int(cond[2:])
	val_inverted = val + 1 if gt else val - 1
	if_cond_is_true = both(ch, gt, val, acceptance_ranges_inner([it.split(":")[1]]))
	if_cond_is_false = both(ch, not gt, val_inverted, acceptance_ranges_inner(w[1:]))
	return if_cond_is_true + if_cond_is_false

p2 = 0
for rng in acceptance_ranges_outer('in'):
	v = 1
	for lo, hi in rng:
		v *= hi - lo + 1
	p2 += v
print(p2)

131192538505367
